# Projeto Fase 6 – Rede Neural YOLO

Este notebook implementa um sistema de visão computacional utilizando **YOLOv5** para detecção de objetos. Como parte da fase 6 do curso, montamos um dataset sintético com duas classes (círculo vermelho e quadrado azul) e seguimos as etapas de preparação, treinamento, validação e teste.

**Estrutura do notebook:**

1. Preparação do dataset (geração de imagens e rótulos)
2. Configuração do arquivo `dataset.yaml`
3. (Opcional) Instalação e configuração do YOLOv5 no Google Colab
4. Treinamento do modelo com diferentes números de épocas
5. Validação e teste
6. Conclusões



In [3]:
import os, random
from PIL import Image, ImageDraw
# Cria a estrutura de pastas
def generate_dataset(base_path, num_train=32, num_val=4, num_test=4):
    os.makedirs(os.path.join(base_path, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'images', 'test'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'val'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'test'), exist_ok=True)
    for split, num in [('train', num_train), ('val', num_val), ('test', num_test)]:
        for i in range(num):
            img, label = create_image()
            img.save(os.path.join(base_path, 'images', split, f'{i}.png'))
            with open(os.path.join(base_path, 'labels', split, f'{i}.txt'), 'w') as f:
                f.write(label)
# Cria uma imagem com um círculo ou um quadrado
def create_image(img_size=256):
    img = Image.new('RGB', (img_size, img_size), 'white')
    draw = ImageDraw.Draw(img)
    class_id = random.choice([0, 1])
    if class_id == 0:
        r = random.randint(20, 50)
        x = random.randint(r, img_size - r)
        y = random.randint(r, img_size - r)
        draw.ellipse([x-r, y-r, x+r, y+r], fill='red')
        width = height = 2*r/img_size
    else:
        side = random.randint(40, 100)
        x = random.randint(side//2, img_size - side//2)
        y = random.randint(side//2, img_size - side//2)
        draw.rectangle([x-side//2, y-side//2, x+side//2, y+side//2], fill='blue')
        width = height = side/img_size
    x_center, y_center = x/img_size, y/img_size
    return img, f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
generate_dataset('dataset')

## Instalação do YOLOv5

Para treinar a YOLOv5 no Google Colab, recomendamos clonar o repositório oficial e instalar os requisitos. No Colab, execute:

```python
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt
```

Em seguida, copie o arquivo de configuração `dataset.yaml` para a pasta raiz do Drive ou para a pasta do repositório e edite o caminho conforme necessário.


In [6]:
# Verificar se dataset.yaml existe
import os
print("Verificando dataset.yaml...")
if os.path.exists('/content/dataset.yaml'):
    print('✓ dataset.yaml encontrado em /content/')
else:
    print('✗ dataset.yaml NÃO encontrado em /content/')

if os.path.exists('/content/yolov5/dataset.yaml'):
    print('✓ dataset.yaml encontrado em /content/yolov5/')
else:
    print('✗ dataset.yaml NÃO encontrado em /content/yolov5/')

print("\nArquivos .yaml na raiz /content/:")
print([f for f in os.listdir('/content/') if f.endswith('.yaml')])

print("\nArquivos .yaml em /content/yolov5/:")
if os.path.exists('/content/yolov5'):
    print([f for f in os.listdir('/content/yolov5/') if f.endswith('.yaml')])

Verificando dataset.yaml...
✗ dataset.yaml NÃO encontrado em /content/
✗ dataset.yaml NÃO encontrado em /content/yolov5/

Arquivos .yaml na raiz /content/:
[]

Arquivos .yaml em /content/yolov5/:
[]


In [7]:
# Criar arquivo dataset.yaml
import yaml

dataset_config = {
    'path': '/content/dataset',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 2,
    'names': ['red_circle', 'blue_square']
}

# Salvar na raiz /content/
with open('/content/dataset.yaml', 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print('✓ dataset.yaml criado em /content/')
print('\nConteúdo:')
with open('/content/dataset.yaml', 'r') as f:
    print(f.read())

✓ dataset.yaml criado em /content/

Conteúdo:
names:
- red_circle
- blue_square
nc: 2
path: /content/dataset
test: images/test
train: images/train
val: images/val



In [5]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17608, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17608 (delta 7), reused 2 (delta 2), pack-reused 17593 (from 2)
Receiving objects: 100% (17608/17608), 16.82 MiB | 17.02 MiB/s, done.
Resolving deltas: 100% (11983/11983), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.6 MB/s eta 0:00:00


In [9]:
# Treinamento com 30 épocas
!python train.py --data /content/dataset.yaml --weights yolov5s.pt --img 256 --epochs 30 --name shape_yolo_30

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-10-10 15:12:42.409601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760109162.435775    6543 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760109162.443444    6543 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [10]:
# Treinamento com 60 épocas
!python train.py --data /content/dataset.yaml --weights yolov5s.pt --img 256 --epochs 60 --name shape_yolo_60

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-10-10 15:19:36.014052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760109576.043061    8284 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760109576.051726    8284 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760109576.076904    8284 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760109576.077025    8284 computation_placer.cc:177] computation placer already registere

## Avaliação e Conclusões

Após o treinamento, os resultados de validação e teste são salvos em `yolov5/runs/train/shape_yolo_30` e `shape_yolo_60`. Compare as métricas de **precisão (P)**, **recall (R)** e **mAP** para as diferentes quantidades de épocas. Observa-se que aumentar o número de épocas tende a melhorar a precisão até certo ponto, mas também aumenta o tempo de treinamento.

Com base nas métricas obtidas, descreva aqui os pontos fortes e limitações do modelo treinado com 30 e 60 épocas, mencionando a variação de desempenho entre as classes círculo e quadrado.
